In [1]:
import os
import requests
import docx
try:
    import win32com.client as win32
    from win32com.client import constants
except ImportError:
    pass
from io import BytesIO

### sometimes cache issues occur - delete it from C:\Users\USER_NAME\AppData\Local\Temp\gen_py
def get_doc_from_url(url):
    try:
        content = requests.get(url).content
        
        if content.startswith(b"PK"):
            # assume it's a docx
            buffer = BytesIO(content)
        else:
            # assume it's a doc
            filepath = os.path.join(os.getcwd(), 'tmp.doc')
            
            with open(filepath, 'wb') as file:
                file.write(content)

            word = win32.gencache.EnsureDispatch('Word.Application')
            doc = word.Documents.Open(filepath)
            doc.Activate()
            word.ActiveDocument.SaveAs(filepath, FileFormat=constants.wdFormatXMLDocument)
            doc.Close(False)
            
            buffer = BytesIO(open(filepath, "rb").read())
            os.remove(filepath)
            
        buffer.seek(0)
        doc = docx.Document(buffer)
        return doc
    except Exception as e:
        print(f"{e} - Cannot get content properly from", url)

In [2]:
import docx
import os
import re
import swifter
import pandas as pd

### Algorithm to extract protocol data ###

relevant_tags = ['דובר-המשך', 'קריאה', 'קריאות', 'יור', 'דובר', 'אורח', 'קריאה', 'דובר_המשך']

def is_speaker(p):
    if is_speaker_strong(p):
        return True
    return p.text.endswith(':') and p.runs[0].underline

def is_speaker_strong(p):
    return p.text and len(p.text.strip()) > 2 and p.style.name in relevant_tags

def process_protocols(df):
    dfs = []
    non_existent = 0
    j = 0
    for i, row in df.iterrows():
        j += 1
        print(f'processing row {j} - {round(j /df.shape[0] * 100, 2)}%, S: {non_existent}                    ', end='\r')

        url = row['FilePath']
        document = get_doc_from_url(url)
        
        running_index = 0
        latest_speaker = ''
        should_record = False
        records = []            
        
        try:
            for p in document.paragraphs:
                if is_speaker(p):
                    should_record = True
                    latest_speaker = p.text
                    continue
                    
                if should_record and len(p.text) > 1:
                    running_index += 1                    
                    records.append({'Index': running_index, 'Speaker': latest_speaker, 'RawText': p.text, '$Type': 'Plenum'})
            
            file_df = pd.DataFrame.from_records(records)
            file_df['DocumentPlenumSessionID'] = row['DocumentPlenumSessionID']
            file_df['StartDate'] = row['StartDate']
            
            if len(file_df):
                dfs.append(file_df)       
        except:
            print(f'Error: {e}', url)
            import traceback
            traceback.print_exc()

    df = pd.concat(dfs, ignore_index=True)
    #df.to_csv('PlenumQuotesSince2015.csv', index=False, encoding='utf-8-sig')
    return df

In [3]:
import sqlalchemy
import urllib
import pyodbc
from sqlalchemy import event

# Set DB connection
params = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=tcp:<SERVER>;PORT=1433;DATABASE=<DATABASE>;UID=<USER_ID>;PWD=<PASSOWRD>'
db_params = urllib.parse.quote_plus(params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))      
df = pd.read_sql(
"""
SELECT s.StartDate, d.DocumentPlenumSessionID, d.FilePath 
FROM KNS_PlenumSession s
JOIN KNS_DocumentPlenumSession d on d.PlenumSessionID = s.PlenumSessionID 
WHERE d.ApplicationID = 1
AND d.GroupTypeID = 28
AND s.StartDate > DATEADD(MONTH, -1, GETDATE())
ORDER BY s.StartDate ASC
""", engine)

In [4]:
quotes = process_protocols(df)
display(quotes)

,Index,Speaker,RawText,$Type,DocumentPlenumSessionID,StartDate
0,1,"<< יור >> היו""ר יריב לוין: << יור >>","חברות וחברי הכנסת, היום יום שני, ב' בניסן תשפ""...",Plenum,598304,2021-03-15 13:00:00
1,2,"<< יור >> היו""ר יריב לוין: << יור >>","הודעה למזכירת הכנסת, בבקשה.",Plenum,598304,2021-03-15 13:00:00
2,3,<< דובר >> מזכירת הכנסת ירדנה מלר-הורוביץ: <<...,"תודה. ברשות יושב-ראש הכנסת, הינני מתכבדת להודי...",Plenum,598304,2021-03-15 13:00:00
3,4,<< דובר >> מזכירת הכנסת ירדנה מלר-הורוביץ: <<...,תקנות סמכויות מיוחדות להתמודדות עם נגיף הקורונ...,Plenum,598304,2021-03-15 13:00:00
4,5,"<< יור >> היו""ר יריב לוין: << יור >>",תודה רבה.,Plenum,598304,2021-03-15 13:00:00
...,...,...,...,...,...,...
235370,634,"<< יור >> היו""ר מיקי לוי: << יור >>",הודעה לסגנית מזכירת הכנסת.,Plenum,622089,2022-03-08 16:00:00
235371,635,<< דובר >> סגנית מזכירת הכנסת אלינור ימין: <<...,"תודה. ברשות יושב-ראש הכנסת, הינני מתכבדת להודי...",Plenum,622089,2022-03-08 16:00:00
235372,636,"<< יור >> היו""ר מיקי לוי: << יור >>",תודה רבה לסגנית מזכירת הכנסת.,Plenum,622089,2022-03-08 16:00:00
235373,637,"<< יור >> היו""ר מיקי לוי: << יור >>","תם סדר-היום. הישיבה הבאה תתקיים מחר, יום רביעי...",Plenum,622089,2022-03-08 16:00:00


In [5]:
import functools
import swifter 
import re

re_remove = re.compile('<<.+?>>|\(.+?\)|היו\"ר|[^\sא-ת]')
re_space = re.compile('\s+')
re_tags = re.compile('<<.+?>>')

def parse_person(p):
    first_name = p.CleanedFirstName
    last_name = p.CleanedLastName
    s = set()
    s.update(first_name.split())
    s.update(last_name.split())
    
    return {
        'first': first_name,
        'first_split': first_name.split(),
        'last': last_name,
        'last_split': last_name.split(),
        'full_name': f'{first_name} {last_name}',
        'parts': s,
    }

@functools.lru_cache(64000)
def name_to_id(name):
    #if name in parsed_name_to_id_cache:
        #return parsed_name_to_id_cache[name]
    
    name_split = name.split()
    
    if len(name_split) < 2 or len(name_split) > 10:
        return None
    
    for id_, parsed in parsed_person.items():
        # exact match (order doesnt matter)?
        if all(map(lambda n: n in parsed['parts'], name_split)):
            return id_
        
    for id_, parsed in parsed_person.items():
        # partial match (last name equals, first name contains at least one piece)
        if parsed['last'] in name_split and any(map(lambda n: n in name_split, parsed['first_split'])):
            return id_
    
    for id_, parsed in parsed_person.items():
        # full name substring
        if parsed['full_name'] in name:
            return id_
    """
    for id_, parsed in parsed_person.items():
        # split name substring
        if parsed['first'] in name and parsed['last'] in name:
            return id_
            """

def clean_speaker(txt):
    try:
        txt = txt.replace('יי', 'י') #in notebook the hebrew looks RTL in the replacement
        txt = txt.replace('וו', 'ו') #in notebook the hebrew looks RTL in the replacement
        txt = txt.replace('אא', 'א') #in notebook the hebrew looks RTL in the replacement
        txt = re_remove.sub('', txt)
        txt = re_space.sub(' ', txt)
        txt = re_tags.sub('', txt)
        txt = txt.strip()
    except:
        print(txt)
        raise
    return txt
    
speakers = quotes["Speaker"].swifter.apply(clean_speaker).unique()

# Load person table
person = pd.read_sql(
"""
SELECT p.PersonID, p.FirstName, p.LastName FROM KNS_Person p 
JOIN KNS_PersonToPosition kptp on kptp.PersonID = p.PersonID
WHERE StartDate > '2000-01-01'
GROUP BY p.PersonID, p.FirstName, p.LastName 
ORDER BY MAX(kptp.StartDate) DESC
""", engine)

person['CleanedFirstName'] = person['FirstName'].swifter.apply(clean_speaker)
person['CleanedLastName'] = person['LastName'].swifter.apply(clean_speaker)

# Map extracted speaker name to person ID
parsed_person = {}
for _, p in person.iterrows():
    parsed_person[p.PersonID] = parse_person(p)

name_mapping = {}    
total = len(speakers)
print('Speakers:', total)

for i, name in enumerate(speakers):
    try:
        name_mapping[name] = name_to_id(name)
    except:
        name_mapping[name] = None
    print(f'{i}, {round((i+1) / total * 100, 2)}%      ', end='\r')

# Apply name mapping
quotes['PersonID'] = quotes['Speaker'].swifter.apply(clean_speaker).apply(lambda x: name_mapping[x])

display(quotes)

Pandas Apply:   0%|          | 0/235375 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/553 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/553 [00:00<?, ?it/s]

Speakers: 307


Pandas Apply:   0%|          | 0/235375 [00:00<?, ?it/s]

,Index,Speaker,RawText,$Type,DocumentPlenumSessionID,StartDate,PersonID
0,1,"<< יור >> היו""ר יריב לוין: << יור >>","חברות וחברי הכנסת, היום יום שני, ב' בניסן תשפ""...",Plenum,598304,2021-03-15 13:00:00,12951.0
1,2,"<< יור >> היו""ר יריב לוין: << יור >>","הודעה למזכירת הכנסת, בבקשה.",Plenum,598304,2021-03-15 13:00:00,12951.0
2,3,<< דובר >> מזכירת הכנסת ירדנה מלר-הורוביץ: <<...,"תודה. ברשות יושב-ראש הכנסת, הינני מתכבדת להודי...",Plenum,598304,2021-03-15 13:00:00,48.0
3,4,<< דובר >> מזכירת הכנסת ירדנה מלר-הורוביץ: <<...,תקנות סמכויות מיוחדות להתמודדות עם נגיף הקורונ...,Plenum,598304,2021-03-15 13:00:00,48.0
4,5,"<< יור >> היו""ר יריב לוין: << יור >>",תודה רבה.,Plenum,598304,2021-03-15 13:00:00,12951.0
...,...,...,...,...,...,...,...
235370,634,"<< יור >> היו""ר מיקי לוי: << יור >>",הודעה לסגנית מזכירת הכנסת.,Plenum,622089,2022-03-08 16:00:00,23632.0
235371,635,<< דובר >> סגנית מזכירת הכנסת אלינור ימין: <<...,"תודה. ברשות יושב-ראש הכנסת, הינני מתכבדת להודי...",Plenum,622089,2022-03-08 16:00:00,134.0
235372,636,"<< יור >> היו""ר מיקי לוי: << יור >>",תודה רבה לסגנית מזכירת הכנסת.,Plenum,622089,2022-03-08 16:00:00,23632.0
235373,637,"<< יור >> היו""ר מיקי לוי: << יור >>","תם סדר-היום. הישיבה הבאה תתקיים מחר, יום רביעי...",Plenum,622089,2022-03-08 16:00:00,23632.0


In [ ]:
import math
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, parallel_bulk
from datetime import datetime

quotes.replace({math.nan: None}, inplace=True)

es = Elasticsearch(["ELASTICSERVER"], http_auth=('USERNAME', 'PASSWORD'))
#res = es.search(index="quotes", body={"query": {"match_all": {}}})

def gen():
    j = 0
    for i, row in quotes.iterrows():
        j = j + 1
        row = dict(row)
        if row["$Type"] == "Committee":    
            row["DocumentID"] = row.pop("DocumentCommitteeSessionID")
        else:
            row["DocumentID"] = row.pop("DocumentPlenumSessionID")
        row["$Timestamp"] = datetime.now()
        row["_index"] = "quotes"
        row["_id"] = f'{row["$Type"]}:{row["DocumentID"]}:{row["Index"]}'
        yield row
        if j % 1000 == 0:
            print(f'{j}          ', end='\r')
        
for success, info in parallel_bulk(es, gen()):
    if not success:
        print('failed', info)
        
es.transport.close()

/var/folders/rb/jgsgx6y50419kr6gwygl985c0000gn/T/ipykernel_38128/878008610.py:8: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([""], http_auth=('elastic', ''))
